<a href="https://colab.research.google.com/github/vattevaii/SIMPLE-CHATBOT-using-tflearn/blob/master/BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
#Define stemming
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Explain libraries
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where are 

In [0]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [0]:
words =[]
classes=[]
documents=[]
ignore = ['?']
# loop through all tags in intents
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # Tokenize all words in pattern 
    w = nltk.word_tokenize(pattern)
    # Add to list of words
    words.extend(w)
    # Add word to the document
    documents.append((w,intent['tag']))
    # Add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [18]:
# Perform stemming and lowercase each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

#Remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), " Documents")
print(len(classes)," Classes")
print(len(words)," Unique words")

31  Documents
9  Classes
57  Unique words


In [0]:
# Create Training 
training = []
output = []
# Array for output
output_empty = [0]*len(classes)
# print(output_empty,"\n")
# create training set
for doc in documents:
  # B-O-W Bags of words
  bag = []
  # List of tokenized words
  pattern_words = doc[0]
  # STEMMING EACH WORD
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # Create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  # Output is 1 for current tag and 0 for other class
  output_row = list(output_empty)
  # print(output_row)

  output_row[classes.index(doc[1])] = 1
  # print(output_row)
  
  training.append([bag, output_row])
# print(training)
#shuffling features
random.shuffle(training)
training = np.array(training)
# for x in training:
  # print(x)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [20]:
print(train_x)

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
print(train_y)

[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0]]


In [22]:
#resetting underlying graph data
tf.reset_default_graph()

#Building Neural Networks
net = tflearn.input_data(shape = [None, len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net , len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#defining model and setting up tensorboard
model = tflearn.DNN(net ,tensorboard_dir='tflearn_logs')

#start training 
model.fit(train_x , train_y, n_epoch=1000 , batch_size = 8 ,show_metric = True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.06132 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.06132 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.06214 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.06214 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [0]:
import pickle
pickle.dump({'words':words,
             'train_x':train_x,
             'train_y':train_y,
             'classes':classes},
             open('training_data','wb'))

In [0]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [0]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [32]:
#loading saved data 
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [0]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))
            results.pop(0)

In [0]:
# for i in intents['intents']:
  # print(i)

In [37]:
response("Where is your base of operation?")

Our speciality for today is Chicken Tikka


In [38]:
classify("Where is your base of operation?")

[('todaysmenu', 0.36242786), ('location', 0.3132405)]

In [41]:
classify("where Locate")

[('location', 0.9163958)]

In [0]:
# SEQUENCE TO SEQUENCE MODEL FOR CHATBOT  -->ONLY TENSORFLOW 1.0